In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [10]:
# Load VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Add custom top layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine the base model with custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of VGG16 base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
# Paths
train_dir = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Train'

# Data augmentation and preprocessing for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 5002 images belonging to 18 classes.


In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20
)


Epoch 1/20


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


156/156 ━━━━━━━━━━━━━━━━━━━━ 892s 6s/step - accuracy: 0.0437 - loss: -563.5696
Epoch 2/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 666us/step - accuracy: 0.1250 - loss: -2637.5664
Epoch 3/20


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


156/156 ━━━━━━━━━━━━━━━━━━━━ 886s 6s/step - accuracy: 0.0486 - loss: -4173.8384
Epoch 4/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 148us/step - accuracy: 0.0312 - loss: -8763.3604
Epoch 5/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 919s 6s/step - accuracy: 0.0482 - loss: -10249.3750
Epoch 6/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 142us/step - accuracy: 0.0938 - loss: -16276.7891
Epoch 7/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 951s 6s/step - accuracy: 0.0463 - loss: -19191.2871
Epoch 8/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 284us/step - accuracy: 0.0000e+00 - loss: -25266.3398
Epoch 9/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 818s 5s/step - accuracy: 0.0490 - loss: -30370.5938
Epoch 10/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 83us/step - accuracy: 0.0312 - loss: -42879.8906 
Epoch 11/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 787s 5s/step - accuracy: 0.0447 - loss: -44394.2109
Epoch 12/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 66us/step - accuracy: 0.1250 - loss: -49032.8672 
Epoch 13/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 783s 5s/step - accuracy: 0.0516 - loss: 

In [22]:
# Path to save the trained model
model_save_path = 'D:/windsor/Sem-2/ASP/Project/Project/saved_model/fruit_veg_classifier.h5'

# Save the model
model.save(model_save_path)

print(f'Model saved to {model_save_path}')
print(f'Accuracy of model is:', accuracy) 

Model saved to D:/windsor/Sem-2/ASP/Project/Project/saved_model/fruit_veg_classifier.h5


In [28]:
import numpy as np
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

# Mapping from class indices to class labels
class_indices = train_generator.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}

def detect_and_classify(image_path):
    # Load and preprocess the image
    img = keras_image.load_img(image_path, target_size=(150, 150))
    x = keras_image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Predict using the model
    preds = model.predict(x)
    class_idx = np.argmax(preds[0])
    class_label = idx_to_class[class_idx]

    return class_label

# Path to the test image
image_path = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Test/p_r023.png'

# Classify the image
result = detect_and_classify(image_path)
print(f'The image at {image_path} is classified as: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
The image at D:/windsor/Sem-2/ASP/Project/Project/dataset/Test/p_r023.png is classified as: freshapples


In [15]:
def classify_objects(image, contours):
    results = []
    for contour in contours:
        # Create a bounding box around each contour
        x, y, w, h = cv2.boundingRect(contour)
        crop_img = image[y:y+h, x:x+w]
        
        # Resize the cropped image to match the input size of VGG16
        crop_img = cv2.resize(crop_img, (150, 150))
        
        # Preprocess the image
        x = image.img_to_array(crop_img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        # Predict using the model
        preds = model.predict(x)
        class_label = 'Rotten' if preds[0] > 0.5 else 'Fresh'
        
        # Store the result
        results.append({'label': class_label, 'bounding_box': (x, y, w, h)})
    
    return results


In [16]:
def display_results(image, results):
    for result in results:
        label = result['label']
        x, y, w, h = result['bounding_box']
        
        # Draw bounding box and label
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Show the image with labels
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()


In [19]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Add custom top layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine the base model with custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of VGG16 base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation and preprocessing for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Path to the training directory
train_dir = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Train'

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)


Found 5002 images belonging to 18 classes.
Epoch 1/10
 51/156 ━━━━━━━━━━━━━━━━━━━━ 10:54 6s/step - accuracy: 0.0529 - loss: -138.2738

KeyboardInterrupt: 